In [19]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [20]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [21]:
subject = 'Acre - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [22]:
data = pd.read_csv('2003_mo_model_input_AC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.722906,0.691320,7.285707e+06,413409.255032,9.318417,6.587220e+06,3.260
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.718028,0.691617,7.294881e+06,413607.843560,9.319990,6.590338e+06,2.191
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.713149,0.691914,7.304055e+06,413806.432089,9.321563,6.593455e+06,2.154
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.708271,0.692212,7.313229e+06,414005.020618,9.323136,6.596572e+06,2.643
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.703393,0.692509,7.322403e+06,414203.609146,9.324709,6.599689e+06,2.546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.748
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.032
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.573
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.493


In [27]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.809794,-2.186463,-1.762689,0.489108,-0.377392,-2.320347
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.812767,-2.149366,-1.742798,0.505628,-0.353105,-2.278259
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.815740,-2.112268,-1.722906,0.522147,-0.328819,-2.236171
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.818713,-2.075170,-1.703015,0.538666,-0.304532,-2.194083
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.821686,-2.038072,-1.683123,0.555185,-0.280246,-2.151995
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.241503,1.365742,1.057869,-1.253684,-1.748677,0.520808
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.236264,1.353890,1.047686,-1.237936,-1.749205,0.506927
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.231025,1.342037,1.037502,-1.222188,-1.749733,0.493046
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.225786,1.330184,1.027319,-1.206440,-1.750261,0.479164


In [28]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      4.105
1      3.898
2      4.209
3      2.994
4      3.720
       ...  
235      NaN
236      NaN
237      NaN
238      NaN
239      NaN
Name: Acre - Consumo de Cimento (t), Length: 240, dtype: float64

In [29]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.809794,-2.186463,-1.762689,0.489108,-0.377392,-2.320347
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.812767,-2.149366,-1.742798,0.505628,-0.353105,-2.278259
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.815740,-2.112268,-1.722906,0.522147,-0.328819,-2.236171
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.818713,-2.075170,-1.703015,0.538666,-0.304532,-2.194083
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.821686,-2.038072,-1.683123,0.555185,-0.280246,-2.151995
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.455828,1.391708,1.134283,-1.537220,-1.492213,0.684545
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.445046,1.388896,1.136306,-1.537199,-1.507297,0.682671
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.434265,1.386083,1.138329,-1.537178,-1.522382,0.680798
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.423484,1.383271,1.140352,-1.537157,-1.537466,0.678924


In [30]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0       4.105
1       3.898
2       4.209
3       2.994
4       3.720
        ...  
157     9.081
158     9.545
159    10.130
160    14.451
161    13.218
Name: Acre - Consumo de Cimento (t), Length: 162, dtype: float64

In [31]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
162,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,1.401921,1.377647,1.144399,-1.537116,-1.567635,0.675177
163,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,1.391140,1.374834,1.146422,-1.537095,-1.582720,0.673304
164,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,1.380359,1.372022,1.148445,-1.537075,-1.597804,0.671430
165,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,1.369578,1.369210,1.150468,-1.537054,-1.612889,0.669557
166,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,1.358796,1.366398,1.152491,-1.537033,-1.627973,0.667683
167,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,1.348015,1.363585,1.154514,-1.537012,-1.643058,0.665810
168,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,1.337234,1.360773,1.156537,-1.536992,-1.658142,0.663936
169,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,1.332312,1.368101,1.154255,-1.522569,-1.665379,0.660106
170,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,1.327391,1.375429,1.151973,-1.508147,-1.672615,0.656276
171,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,1.322469,1.382758,1.149691,-1.493724,-1.679852,0.652446


In [32]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    13.552
163    15.826
164    14.869
165    14.734
166    12.754
167    11.925
168    13.114
169     8.920
170     9.724
171     9.737
172    10.136
173    14.403
174    12.871
175    15.749
176    14.269
177    15.367
178    14.595
179     9.811
180    11.188
181    10.381
182     8.080
183    11.154
184    12.508
185    12.126
186    14.496
187    16.723
188    15.253
189    16.531
190    15.206
191    10.160
Name: Acre - Consumo de Cimento (t), dtype: float64

In [33]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [34]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [37]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [38]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1470492430, 3535059999, 2426746008, 1302159539, 780429323, 318268618, 2237158122, 1696822316, 2390657211, 3239919337, 2865220032, 3492085175, 2967444398, 2299668568, 3696945475, 3668881464, 919651637, 145320690, 39894175, 1273267315, 2704852805, 1296281065, 1211710277, 719164187, 620834364]


Step: 0 ___________________________________________
val_loss: 3.107177972793579
winner_seed: 1470492430


Step: 1 ___________________________________________
val_loss: 3.6739182472229004


Step: 2 ___________________________________________
val_loss: 3.885237216949463


Step: 3 ___________________________________________
val_loss: 4.519416332244873


Step: 4 ___________________________________________
val_loss: 2.6363794803619385
winner_seed: 780429323


Step: 5 ___________________________________________
val_loss: 1.7521650791168213
winner_seed: 318268618


Step: 6 ___________________________________________
val_loss: 3.4414353370666504


Step: 7 ___________________________________________
val_l

In [39]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 0s 26ms/step - loss: 103.0738 - val_loss: 14.9764
Epoch 2/10000
5/5 [==============================] - 0s 5ms/step - loss: 19.1457 - val_loss: 9.7324
Epoch 3/10000
5/5 [==============================] - 0s 4ms/step - loss: 8.3408 - val_loss: 6.5032
Epoch 4/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.0096 - val_loss: 5.0190
Epoch 5/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.6231 - val_loss: 5.0457
Epoch 6/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.9106 - val_loss: 6.3340
Epoch 7/10000
5/5 [==============================] - 0s 5ms/step - loss: 9.1438 - val_loss: 9.7627
Epoch 8/10000
5/5 [==============================] - 0s 5ms/step - loss: 9.0430 - val_loss: 5.1907
Epoch 9/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.1725 - val_loss: 9.5578
Epoch 10/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.1511 - val_loss: 6.9934
Epoc

Epoch 83/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.2780 - val_loss: 4.2472
Epoch 84/10000
5/5 [==============================] - 0s 4ms/step - loss: 6.1924 - val_loss: 4.1943
Epoch 85/10000
5/5 [==============================] - 0s 4ms/step - loss: 6.1466 - val_loss: 5.5470
Epoch 86/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.0502 - val_loss: 4.5575
Epoch 87/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.1329 - val_loss: 4.8883
Epoch 88/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.9176 - val_loss: 7.1548
Epoch 89/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.9934 - val_loss: 4.5332
Epoch 90/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.8280 - val_loss: 4.5209
Epoch 91/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.5990 - val_loss: 4.5134
Epoch 92/10000
5/5 [==============================] - 0s 4ms/step - loss: 6.1676 - val_loss: 7.5850


5/5 [==============================] - 0s 4ms/step - loss: 5.9899 - val_loss: 5.0463
Epoch 165/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.4675 - val_loss: 4.3206
Epoch 166/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.2769 - val_loss: 4.5692
Epoch 167/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.5198 - val_loss: 6.2479
Epoch 168/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.9517 - val_loss: 4.8735
Epoch 169/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.6982 - val_loss: 5.0749
Epoch 170/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.0801 - val_loss: 5.1109
Epoch 171/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.8978 - val_loss: 4.6558
Epoch 172/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.5936 - val_loss: 5.3996
Epoch 173/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.5727 - val_loss: 4.7206
Epoch 

5/5 [==============================] - 0s 4ms/step - loss: 4.8910 - val_loss: 4.7959
Epoch 246/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.3640 - val_loss: 4.8979
Epoch 247/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.9446 - val_loss: 5.0433
Epoch 248/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.1173 - val_loss: 4.7502
Epoch 249/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.8586 - val_loss: 4.9034
Epoch 250/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.8690 - val_loss: 5.5634
Epoch 251/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.9509 - val_loss: 5.0418
Epoch 252/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.1548 - val_loss: 5.7662
Epoch 253/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.6831 - val_loss: 5.1950
Epoch 254/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.0323 - val_loss: 4.9136
Epoch 

5/5 [==============================] - 0s 5ms/step - loss: 4.5710 - val_loss: 5.8759
Epoch 327/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.8128 - val_loss: 5.6532
Epoch 328/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.5840 - val_loss: 5.1318
Epoch 329/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.6785 - val_loss: 5.6412
Epoch 330/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.8993 - val_loss: 4.8246
Epoch 331/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.6495 - val_loss: 5.0801
Epoch 332/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.7567 - val_loss: 4.8002
Epoch 333/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.4898 - val_loss: 6.5456
Epoch 334/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.4915 - val_loss: 4.7697
Epoch 335/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.8260 - val_loss: 5.0099
Epoch 

...

5/5 [==============================] - 0s 5ms/step - loss: 1.5066 - val_loss: 2.5474
Epoch 1295/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.6133 - val_loss: 3.2008
Epoch 1296/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.0796 - val_loss: 2.2484
Epoch 1297/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.4914 - val_loss: 2.2161
Epoch 1298/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.4538 - val_loss: 3.1366
Epoch 1299/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.5189 - val_loss: 3.0957
Epoch 1300/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.6437 - val_loss: 3.3577
Epoch 1301/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.6006 - val_loss: 2.0694
Epoch 1302/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.6466 - val_loss: 2.9735
Epoch 1303/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.6538 - val_loss: 3.29

5/5 [==============================] - 0s 5ms/step - loss: 1.5290 - val_loss: 3.5980
Epoch 1375/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.3695 - val_loss: 2.6822
Epoch 1376/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.7789 - val_loss: 2.8137
Epoch 1377/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.2214 - val_loss: 2.3507
Epoch 1378/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.4020 - val_loss: 3.0546
Epoch 1379/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.4375 - val_loss: 2.7669
Epoch 1380/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.3786 - val_loss: 3.1090
Epoch 1381/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.4616 - val_loss: 2.9770
Epoch 1382/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.4002 - val_loss: 2.7421
Epoch 1383/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.7139 - val_loss: 2.61

5/5 [==============================] - 0s 5ms/step - loss: 1.6097 - val_loss: 3.8856
Epoch 1455/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.3186 - val_loss: 2.4976
Epoch 1456/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.6955 - val_loss: 2.6381
Epoch 1457/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.2902 - val_loss: 2.4470
Epoch 1458/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.3257 - val_loss: 2.4617
Epoch 1459/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.6352 - val_loss: 2.6180
Epoch 1460/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.2845 - val_loss: 3.1085
Epoch 1461/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.5652 - val_loss: 2.5617
Epoch 1462/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.5400 - val_loss: 3.3635
Epoch 1463/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.5722 - val_loss: 2.88

5/5 [==============================] - 0s 4ms/step - loss: 1.7745 - val_loss: 2.5128
Epoch 1535/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.2721 - val_loss: 2.6586
Epoch 1536/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.1284 - val_loss: 2.8835
Epoch 1537/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.3959 - val_loss: 3.0854
Epoch 1538/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.3704 - val_loss: 3.6935
Epoch 1539/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.3244 - val_loss: 3.4800
Epoch 1540/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.4523 - val_loss: 2.9923
Epoch 1541/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.2717 - val_loss: 2.9943
Epoch 1542/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.2737 - val_loss: 2.6656
Epoch 1543/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.8736 - val_loss: 2.76

5/5 [==============================] - 0s 4ms/step - loss: 1.2021 - val_loss: 2.6343
Epoch 1615/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.3298 - val_loss: 3.0334
Epoch 1616/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.2583 - val_loss: 2.6115
Epoch 1617/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.3151 - val_loss: 2.5844
Epoch 1618/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.5389 - val_loss: 3.3606
Epoch 1619/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.3817 - val_loss: 2.9062
Epoch 1620/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.3690 - val_loss: 2.6610
Epoch 1621/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.2364 - val_loss: 3.2451
Epoch 1622/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.1602 - val_loss: 3.0116
Epoch 1623/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.2649 - val_loss: 2.25

In [40]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [41]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 13ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,11.008513,13.17713,14.26413,14.214443,13.396587,12.705309,9.564781,11.451513,12.790689,12.763729,9.716542,14.379889,11.439791,11.227578,12.658968,13.759941,14.491928,12.68614,11.371549,12.286266,12.615402,12.295727,11.95167,11.546955,12.025164,13.740879,12.266563,11.722494,15.734999,17.099911
Target,13.552,15.826,14.869,14.734,12.754,11.925,13.114,8.92,9.724,9.737,10.136,14.403,12.871,15.749,14.269,15.367,14.595,9.811,11.188,10.381,8.08,11.154,12.508,12.126,14.496,16.723,15.253,16.531,15.206,10.16
Error,2.543487,2.64887,0.604871,0.519557,0.642588,0.780309,3.549219,2.531513,3.066689,3.026729,0.419457,0.023111,1.43121,4.521421,1.610032,1.607059,0.103072,2.87514,0.183549,1.905267,4.535402,1.141726,0.556331,0.579045,2.470837,2.982121,2.986437,4.808506,0.528998,6.939911


In [42]:
display(mae)
display(mape)

2.0707488

0.17318097

In [43]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [44]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[149.43324]] - Target[149.694]| =  Error: [[0.26075745]]; MAPE:[[0.00174194]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[148.33192]] - Target[148.09900000000002]| =  Error: [[0.23292542]]; MAPE:[[0.00157277]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[82.59001]] - Target[88.369]| =  Error: [[5.7789917]]; MAPE:[[0.06539614]]


[array([[0.26075745]], dtype=float32),
 array([[0.23292542]], dtype=float32),
 array([[5.7789917]], dtype=float32)]

2.0908916

0.022903614